<a href="https://colab.research.google.com/github/2005WUYU/file/blob/main/GAT_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 1. 原始数据准备模块 (MRI → FreeSurfer → 图数据)

此模块负责将原始的医学影像数据转化为模型可直接处理的图结构数据。

### 1.1 MRI 采集与 FreeSurfer 处理子模块

* **功能：** 将原始的结构性 T1 加权 MRI 图像（如 `.nii`, `.mgz` 格式）处理成标准化的皮层表面数据。
* **核心技术：**
    * **FreeSurfer `recon-all` 命令：** 这是主要的自动化处理工具。它执行一系列复杂的步骤，包括：
        * **颅骨剥离 (Skull Stripping)：** 移除颅骨和非大脑组织。
        * **强度归一化 (Intensity Normalization)：** 统一图像强度，减少扫描仪差异。
        * **皮层下结构分割 (Subcortical Segmentation)：** 自动分割深层灰质（如丘脑、基底节）和白质。
        * **皮层表面重建 (Cortical Surface Reconstruction)：** 生成白质表面和软脑膜（pial）表面，通常是三角网格。
        * **皮层厚度计算 (Cortical Thickness Calculation)：** 测量每个顶点处的皮层厚度。
        * **表面曲率和脑沟深度计算 (Curvature and Sulcal Depth Calculation)：** 提取局部几何特征。
        * **皮层分 parcellation (Cortical Parcellation)：** 将皮层表面划分为解剖学或功能学区域（如 Desikan-Killiany 或 Destrieux 图谱）。
    * **自动化脚本：** 建议编写 shell 脚本或 Python 包装器来自动化 `recon-all` 的执行和错误处理。
* **关键输出：**
    * `surf/lh.pial`, `rh.pial`: 左右半球的**大脑皮层三角网格**文件。每个文件包含**顶点坐标**（3D 空间位置，例如 `vx, vy, vz`）和**拓扑结构**（顶点之间的连接关系，通常是三角形的索引列表）。
    * `surf/lh.curv`, `lh.sulc`, `lh.thickness`: 每个顶点对应的**曲率值**、**脊谷值**和**皮层厚度值**。
    * `label/lh.aparc.annot`: 每个顶点所属的 **Desikan-Killiany 图谱标签**。
    * `sphere.reg`, `fsaverage`: 用于**跨被试配准**的标准模板。
* **对接要求：**
    * **输入：** 接受 `.nii` 或 `.mgz` 格式的 T1 加权 MRI 图像路径。
    * **输出：** 在指定目录下生成 FreeSurfer 标准输出结构，特别是上述列出的关键文件。确保输出路径可被后续子模块访问。

### 1.2 图结构构建子模块

* **功能：** 将 FreeSurfer 生成的皮层表面数据转化为 PyTorch Geometric (PyG) 框架下的图数据对象。
* **核心技术：**
    * **节点定义：** FreeSurfer 网格中的每个**顶点**被定义为图中的一个**节点**。
    * **边定义：** 网格中**相邻的顶点**之间形成**边**。这通常通过解析 `.pial` 文件中的三角形信息来确定（共享同一条边的两个顶点被视为相邻）。构建**稀疏邻接矩阵**表示这种连接。
    * **节点特征 ($\mathbf{x}$) 构建：**
        * **三维坐标** $(x, y, z)$：直接从 `.pial` 文件中提取。
        * **曲率值 (`curv`)**：从 `lh.curv`/`rh.curv` 文件中读取。
        * **脊谷值 (`sulc`)**：从 `lh.sulc`/`rh.sulc` 文件中读取。
        * **皮层厚度 (`thickness`)**：从 `lh.thickness`/`rh.thickness` 文件中读取。
        * **可选特征：**
            * **节点度数：** 通过邻接矩阵计算每个节点的邻居数量。
            * **平均法向：** 根据邻近三角形的法向量计算。
            * **局部曲率张量：** 更复杂的几何描述符。
        * **特征归一化：** 对所有数值节点特征（除坐标外，或对坐标进行中心化）进行 **Z-score 归一化**（减均值除以标准差）或 **Min-Max 归一化**（缩放到 [0, 1] 区间），以确保不同特征尺度的一致性，有助于模型训练稳定性。
    * **边特征 (`edge_attr`) 构建 (可选，但推荐)：**
        * **点间欧氏距离：** 连接两节点的空间距离。
        * **法向夹角：** 连接两节点的法向量之间的夹角（需要预先计算或从FreeSurfer输出中获取法向量）。
        * **曲率差：** 连接两节点曲率值的绝对差。
        * **局部拉普拉斯张量：** 捕捉局部几何形变的更复杂特征。
        * **特征归一化：** 对所有数值边特征进行归一化。
    * **PyG Data 对象创建：** 使用 `torch_geometric.data.Data` 类封装所有信息。

In [ ]:
from torch_geometric.data import Data
        import torch

        # 示例：假设您已经有了这些数据
        # num_nodes = ...
        # feature_dim = ...
        # num_edges = ...
        # edge_feature_dim = ...

        # node_features_tensor 示例：torch.randn(num_nodes, feature_dim)
        # edge_index_tensor 示例：torch.randint(0, num_nodes, (2, num_edges))
        # edge_features_tensor 示例：torch.randn(num_edges, edge_feature_dim)

        data = Data(
            x=node_features_tensor,         # shape: [num_nodes, feature_dim]
            edge_index=edge_index_tensor,   # shape: [2, num_edges]
            edge_attr=edge_features_tensor  # shape: [num_edges, edge_feature_dim] (如果使用)
        )

* **关键输出：**
    * 每个被试的 **PyG `Data` 对象**。
    * 建议将每个 `Data` 对象保存为独立的 **`.pt` 文件**（例如 `sub-01_lh_pial.pt`），以便高效地加载和训练。
* **对接要求：**
    * **输入：** 接收 1.1 子模块输出的 FreeSurfer 关键文件路径（`.pial`, `.curv`, `.sulc`, `.thickness` 等）。
    * **输出：** 生成一系列 `.pt` 文件，每个文件包含一个 PyG `Data` 对象。文件命名应包含被试 ID 和半球信息，便于索引。

---

## 2. 模型架构与训练模块 (GAT-GAN)

此模块定义并实现 GAT-GAN 模型，负责模型的训练过程。

### 2.1 生成器 G：GAT-Unet 结构子模块

* **功能：** 学习从包含异常的图数据到“健康”图数据的映射。
* **核心技术：**
    * **GATConv 层：** 每个 GATConv 层执行图卷积，并通过**注意力机制**加权邻居特征。这允许模型在聚合信息时动态地关注重要的邻居。

In [ ]:
from torch_geometric.nn import GATConv
        import torch.nn as nn

        class GATConvBlock(nn.Module):
            def __init__(self, in_channels, out_channels, heads=1):
                super().__init__()
                self.conv = GATConv(in_channels, out_channels, heads=heads, dropout=0.2) # 可调dropout
                self.norm = nn.InstanceNorm1d(out_channels * heads) # InstanceNorm
                self.relu = nn.LeakyReLU(0.2) # 激活函数

            def forward(self, x, edge_index, edge_attr=None): # edge_attr 可选
                x = self.conv(x, edge_index)
                x = self.norm(x)
                return self.relu(x)

* **Encoder (下采样路径)：** 由多个 GATConvBlock 堆叠而成，逐渐提取多尺度结构特征。
        * 每个编码层后通常连接**图池化层**，以减少节点数量并扩大感受野。
        * **Graph Pooling：**
            * **TopKPooling：** 基于学习到的分数选择 Top-K 个节点保留。

In [ ]:
from torch_geometric.nn import TopKPooling
                # pool = TopKPooling(in_channels, ratio=0.5)
                # x, edge_index, edge_attr, batch, perm, score = pool(x, edge_index, edge_attr, batch)

* **SAGPooling (Self-Attention Graph Pooling)：** 利用自注意力机制学习每个节点的重要性来选择节点。

In [ ]:
from torch_geometric.nn import SAGPooling
                # pool = SAGPooling(in_channels, ratio=0.5)
                # x, edge_index, edge_attr, batch, perm, score = pool(x, edge_index, edge_attr, batch)

* **记录 `perm` 和 `score`：** 池化层通常会返回被保留节点的索引 (`perm`) 和它们的重要性分数 (`score`)，这些在反池化或解释时可能有用。
    * **Decoder (上采样路径)：** 对应编码器路径，通过上采样恢复原始节点数量，并结合跳跃连接。
        * **上采样：**
            * **MLP 插值：** 最简单的上采样方式，直接使用 MLP 提升特征维度，然后广播到原始节点。
            * **特征广播：** 将池化后的特征广播回原始节点位置。
            * **Graph Unpooling 层：** 与池化层配对，利用池化时的信息（如 `perm`）来恢复图结构和节点数量。
        * **跳跃连接 (Skip Connections)：** 将编码器对应层的特征直接传给解码器，有助于保留细节信息并缓解梯度消失。
* **关键输出：** 一个与输入图结构相同（`edge_index` 不变），但节点特征 $\mathbf{x}$ 被重建为“健康”状态的 PyG `Data` 对象。
* **对接要求：**
    * **输入：** PyG `Data` 对象 (包含 `x`, `edge_index`, `edge_attr`)。
    * **输出：** 与输入具有相同 `edge_index` 和 `num_nodes` 的 PyG `Data` 对象，但其 `x` 属性为重建的“健康”节点特征。

### 2.2 判别器 D：SpectralNorm GCN（PatchGAN 结构）子模块

* **功能：** 区分真实的“健康”图和生成器生成的“健康”图，侧重于局部结构真实性。
* **核心技术：**
    * **GCNConv 层：** 执行基本的图卷积操作，聚合邻居特征。

In [ ]:
from torch_geometric.nn import GCNConv
        import torch.nn as nn

        class GCNConvBlock(nn.Module):
            def __init__(self, in_channels, out_channels):
                super().__init__()
                self.conv = GCNConv(in_channels, out_channels)
                # SpectralNorm applied to conv.weight (or other learnable parameters)
                # This needs to be handled via a hook or by custom layer if not directly supported by PyG's GCNConv
                self.norm = nn.InstanceNorm1d(out_channels) # InstanceNorm 也可以用于判别器
                self.relu = nn.LeakyReLU(0.2)

            def forward(self, x, edge_index, edge_attr=None):
                x = self.conv(x, edge_index)
                x = self.norm(x)
                return self.relu(x)

* **谱归一化 (Spectral Normalization, SN)：** 关键在于稳定 GAN 训练。它限制了判别器权重矩阵的谱范数，从而控制了判别器的 Lipschitz 常数。在 PyTorch 中，可以使用 `torch.nn.utils.spectral_norm` 包裹 GCNConv 层的权重。

In [ ]:
# Example of applying spectral norm (might need custom layer or direct application for GCNConv)
        # self.conv = torch.nn.utils.spectral_norm(GCNConv(in_channels, out_channels))

* **PatchGAN 结构：** 判别器不输出单个真假概率，而是输出一个**局部真实性评分矩阵**。
        * **实现方式：**
            * **方法 1：随机采样子图：** 在训练时，从输入图（真实或生成）中随机选择多个子图。每个子图通过 GCNConv 提取特征，然后通过一个小的 MLP 或线性层输出一个评分。最终的判别器损失是这些子图评分的平均值。
            * **方法 2：基于聚类或区域图谱划分：**
                1.  利用 FreeSurfer 的 `label/lh.aparc.annot` 文件，将节点归类到预定义的脑区（如 Desikan-Killiany 的 68 个区域）。
                2.  对于每个脑区，可以对其内部的节点特征进行聚合（例如，平均池化或基于GCN的读出），得到该区域的特征表示。
                3.  这些区域特征再通过一个小的 MLP 或线性层，为每个区域输出一个真实性评分。
                这种方法确保了“Patch”具有生物学意义。
* **关键输出：** 判别器对输入图的**局部真实性评分矩阵**。
* **对接要求：**
    * **输入：** 一个 PyG `Data` 对象（可以是真实图或生成器 G 的输出图）。
    * **输出：** 一个 Tensor，表示图中各个“Patch”或区域的真实性得分。这个 Tensor 的形状取决于 PatchGAN 的实现方式（例如，`[num_patches, 1]`）。

### 2.3 损失函数与训练机制子模块

* **功能：** 定义 GAN 的总损失函数，并管理生成器和判别器的训练循环。
* **核心技术：**
    * **总损失构成：**
        
        * **$L_{adv}$ (对抗损失)：**
            * **Wasserstein Loss (WGAN-GP)：** 旨在最小化真实分布和生成分布之间的 Wasserstein 距离。
                * 判别器损失：$D(x_{real}) - D(G(z))$。
                * 生成器损失：$-D(G(z))$。
            * **梯度惩罚 (Gradient Penalty, GP)：** 为了强制判别器的 1-Lipschitz 条件，对真实样本和生成样本之间的插值点计算梯度惩罚。

$L_{total} = L_{adv} + \lambda_1 \cdot L_{recon} + \lambda_2 \cdot L_{spec}$

$L_{total} = L_{adv} + \lambda_1 \cdot L_{recon} + \lambda_2 \cdot L_{spec}$



In [ ]:
# 计算梯度惩罚伪代码
                # alpha = torch.rand(batch_size, 1, 1).to(device)
                # interpolated_samples = alpha * real_samples + (1 - alpha) * fake_samples
                # interpolated_samples.requires_grad_(True)
                # d_interpolated = discriminator(interpolated_samples)
                # gradients = torch.autograd.grad(outputs=d_interpolated, inputs=interpolated_samples,
                #                                 grad_outputs=torch.ones_like(d_interpolated),
                #                                 create_graph=True, retain_graph=True)[0]
                # gradients_norm = gradients.norm(2, dim=1)
                # gradient_penalty = ((gradients_norm - 1) ** 2).mean()
                # D_loss = (fake_score - real_score) + lambda_gp * gradient_penalty

* **$L_{recon}$ (重建损失)：**
            * **L1 损失：** 衡量生成器重建出的“健康”图节点特征 $x_{recon}$ 与原始输入图节点特征 $x_{input}$ 之间的逐点差异。对异常区域惩罚更大。
                $$L_{recon} = || x_{recon} - x_{input} ||_1$$

In [ ]:
# reconstruction_loss = F.l1_loss(x_recon, x_input)

* **$L_{spec}$ (谱距离损失)：**
            * **图拉普拉斯矩阵计算：**
                * 对于无向图，通常使用对称归一化拉普拉斯矩阵：$L_{sym} = I - D^{-1/2} A D^{-1/2}$。
                * 需要从 `edge_index` 构建邻接矩阵 $A$ 和度矩阵 $D$。

In [ ]:
from torch_geometric.utils import get_laplacian
                # 对于 PyG Data 对象，可以方便地获取拉普拉斯矩阵的边索引和值
                edge_index_lap, edge_weight_lap = get_laplacian(edge_index, normalization='sym')
                # 然后需要将稀疏表示转换为稠密矩阵（如果节点数不大）或使用稀疏矩阵库
                # laplacian_matrix_real = to_dense_adj(edge_index_lap, edge_attr=edge_weight_lap)[0]

* **特征值提取：**
                * `torch.linalg.eigvalsh(L)`：计算对称矩阵的所有特征值。对于节点数大的图，直接计算会很慢。
                * **近似方法 (如 Lanczos 算法)：** 对于大型稀疏矩阵，使用 `scipy.sparse.linalg.eigsh` 或 PyTorch 中实现的近似方法（如果可用）来高效提取前 k 个特征值。
            * **损失计算：** 衡量真实图和生成图拉普拉斯矩阵前 k 个特征值之间的 L2 距离。
                $$L_{spec} = || eig_{real}[:k] - eig_{fake}[:k] ||_2$$

In [ ]:
# eig_real_k = torch.linalg.eigvalsh(laplacian_matrix_real)[:k]
                # eig_fake_k = torch.linalg.eigvalsh(laplacian_matrix_fake)[:k]
                # spectral_loss = F.mse_loss(eig_real_k, eig_fake_k) # L2距离

* **优化器与训练循环：**
        * 使用 Adam 或 RMSprop 等优化器。
        * **交替训练：**
            1.  **训练判别器 D (多步)：**
                * 从数据集中获取真实图。
                * 生成器生成虚假图。
                * 计算判别器在真实图和虚假图上的得分。
                * 计算 $L_{adv}$ 的判别器部分，包括梯度惩罚。
                * 更新判别器参数。
            2.  **训练生成器 G (一步)：**
                * 生成器生成虚假图。
                * 计算判别器在虚假图上的得分。
                * 计算 $L_{adv}$ 的生成器部分 ($-D(G(z))$)。
                * 计算 $L_{recon}$ 和 $L_{spec}$。
                * 将三者加权求和，计算总生成器损失。
                * 更新生成器参数。
        * **定期评估和保存模型：** 在验证集上评估模型性能，并保存表现最佳的模型权重。
* **关键输出：** 训练好的生成器 G 和判别器 D 的模型权重文件（`.pt` 或 `.pth` 格式）。
* **对接要求：**
    * **输入：** 训练数据集（`.pt` 格式的 PyG `Data` 对象列表），以及生成器 G 和判别器 D 的模型定义。
    * **输出：** 训练好的模型权重文件。

---

## 3. 测试与异常评分模块

此模块负责使用训练好的模型对新数据进行推理，并计算异常评分。

### 3.1 推理子模块

* **功能：** 将训练好的生成器 G 应用于新的、疑似含有异常的输入图，生成其“健康”重建图。
* **核心技术：**
    * 加载训练好的生成器 G 模型权重。
    * 将预处理后的输入图 (PyG `Data` 对象) 送入生成器。
    * 生成器输出重建的“健康”图。
* **关键输出：**
    * 输入图的节点特征 ($\mathbf{x}_{input}$)。
    * 生成器重建的“健康”图的节点特征 ($\mathbf{x}_{recon}$)。
    * 原始图的拓扑结构 (`edge_index`) 和边特征 (`edge_attr`)。
* **对接要求：**
    * **输入：** 待检测的单个或批量的 PyG `Data` 对象，以及已加载的生成器 G 模型。
    * **输出：** 对应的重建后的 PyG `Data` 对象（只改变 `x` 属性）。

### 3.2 异常评分计算子模块

* **功能：** 基于推理结果计算节点、区域和全脑级别的异常评分。
* **核心技术：**
    * **节点级别异常得分：**
        * **Score_L1** $= || \mathbf{x}_{input} - \mathbf{x}_{recon} ||_1$：逐节点计算输入特征与重建特征之间的 L1 距离。
        * **Score_spec** $= || eig_{input}[:k] - eig_{recon}[:k] ||_2$：
            * 重新计算输入图和重建图的拉普拉斯矩阵。
            * 提取各自的前 k 个特征值。
            * 计算特征值之间的 L2 距离。
        * **Score_adv** $= - D(\mathbf{x}_{recon})$：
            * 将重建图输入判别器 D（需要加载训练好的判别器模型）。
            * 获取判别器的输出评分，并取负值。负值越大，表示判别器认为重建图越不真实，从而间接反映原始图的异常程度。
            * 或者，可以计算判别器不同层在真实图和生成图上的**特征表示差异**（例如，L2 距离），这可以提供更丰富的鉴别信息。
    * **区域/脑区级别异常得分：**
        * 利用 1.1 子模块输出的 FreeSurfer **`label/lh.aparc.annot` 文件**，将每个节点映射到其所属的 Desikan-Killiany 区域。
        * 对于每个区域，将该区域内所有节点的**节点级别异常得分进行平均**（或其他统计量如中位数、最大值）作为该区域的异常得分。
    * **全脑级异常得分：**
        * 对所有节点的节点级别异常得分进行**统计汇总**（如均值、标准差、最大值或高于某个阈值的节点比例），得到一个代表全脑异常程度的单一数值。
    * **评分融合策略：**
        * **加权组合：** `Score_total = α * Score_L1 + β * Score_spec + γ * Score_adv`
        * **参数 ($\alpha, \beta, \gamma$) 设定：** 可以通过在验证集上进行**交叉验证**来优化这些权重，以最大化异常检测的敏感度和特异性；或者通过**专家经验**进行初步设定。
* **关键输出：**
    * 每个节点的 L1、谱距离和对抗性异常得分。
    * 每个 Desikan-Killiany 脑区的异常得分。
    * 全脑异常总得分。
* **对接要求：**
    * **输入：** 推理子模块的输出 ($\mathbf{x}_{input}$, $\mathbf{x}_{recon}$, `edge_index`)，以及 FreeSurfer 的 `.annot` 文件路径和训练好的判别器 D 模型。
    * **输出：** 一个数据结构（例如 Pandas DataFrame 或字典），包含不同级别（节点、区域、全脑）的异常得分。

---

## 4. 图谱映射与可视化模块

此模块负责将异常评分映射回大脑皮层表面，并生成直观的可视化结果。

### 4.1 映射与配准子模块

* **功能：** 将计算出的异常得分映射到标准的大脑表面模板上，并确保不同个体之间的可比性。
* **核心技术：**
    * **FreeSurfer 图谱标签 (annot 文件)：** 每个节点已经带有 Desikan-Killiany 图谱标签，这可以直接用于将节点级得分聚合到区域级，并在可视化时用区域着色。
    * **跨被试配准 (`sphere.reg`, `fsaverage`)：**
        * FreeSurfer 的 `sphere.reg` 文件包含了将个体大脑皮层表面对齐到标准 `fsaverage` 球面模板所需的形变信息。
        * 在可视化时，通常会将个体计算出的节点级异常得分**投影到 `fsaverage` 模板的顶点上**，这样所有被试的异常模式都可以在一个共同的参考系统下进行比较和叠加。
    * **数据结构转换：** 将 PyG 中的节点 ID 与 FreeSurfer 的顶点 ID 对齐，确保得分能正确映射回表面。
* **关键输出：**
    * 在 `fsaverage` 标准模板上每个顶点（或每个区域）的异常得分。
* **对接要求：**
    * **输入：** 异常评分计算子模块输出的节点级和区域级异常得分，以及 FreeSurfer 的 `lh.aparc.annot` 和 `sphere.reg` 文件。
    * **输出：** 包含每个 `fsaverage` 顶点或区域异常得分的数据结构。

### 4.2 可视化输出子模块

* **功能：** 生成各种直观的大脑异常可视化图像，便于临床医生和研究人员理解。
* **核心技术：**
    * **绘制热力图 (Vertex Color Mapping)：**
        * 在重建或标准化的皮层表面上，根据每个顶点的异常得分进行颜色编码（热力图），得分越高颜色越突出（例如，从蓝色到红色）。
        * 可以使用颜色条指示得分范围。
    * **区域颜色标注 (Region-based Coloring)：**
        * 根据区域级异常得分，对不同的脑区进行整体颜色填充。
        * 对于 Desikan-Killiany 图谱，可以为每个区域分配一个颜色，表示其异常程度。
    * **疑似病灶定位 (Top-k Anomaly Regions)：**
        * 突出显示异常得分最高的 Top-k 区域或顶点，用特定标记（如边框、高亮）进行标注。
    * **交互式可视化：**
        * **`vedo` / `pyvista`：** 强大的 Python 库，提供交互式 3D 渲染，允许用户旋转、缩放、平移大脑模型，并动态调整热力图阈值。
        * 支持加载 `.ply`, `.obj` 等网格格式，并将颜色数据映射到顶点。
    * **静态可视化：**
        * **`nilearn.plot_surf`：** 专门用于神经影像数据的绘图库，特别适合绘制 `fsaverage` 模板上的表面图。
        * **`matplotlib` + `mayavi`：** 结合使用可以生成高质量的 3D 静态图像，但 `mayavi` 的安装和配置可能相对复杂。
* **关键输出：**
    * 高质量的**热力图图像**（PNG, SVG 等格式）。
    * **区域颜色标注图**。
    * **交互式 3D 可视化界面或文件**（例如 HTML 嵌入的交互式图）。
* **对接要求：**
    * **输入：** 映射与配准子模块输出的 `fsaverage` 模板上的异常得分，以及相关的 FreeSurfer 表面文件（如 `fsaverage/surf/lh.pial`）。
    * **输出：** 可视化图像文件和/或交互式可视化对象。
